<a href="https://colab.research.google.com/github/Emmanuel-K07/chatterbox/blob/master/stable/526_mix_comfyui_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ComfyUI Installation for Google Colab - Version corrigée
# Installation avec tunnel Cloudflare fonctionnel

import subprocess
import sys
import os
from pathlib import Path
import requests
import time

def run_cmd(cmd, check=True, capture=False):
    """Execute command with proper error handling"""
    print(f"🔧 Exécution: {cmd[:80]}...")
    result = subprocess.run(cmd, shell=True, capture_output=capture, text=True, executable='/bin/bash')
    if check and result.returncode != 0:
        print(f"❌ Erreur lors de: {cmd}")
        if capture and result.stderr:
            print(f"Erreur: {result.stderr}")
        return False
    return result.stdout if capture else True

print("🚀 Installation ComfyUI pour Google Colab")
print("=" * 50)

# 1. Préparation environnement
print("📦 Préparation de l'environnement...")
os.chdir('/content')

# 2. Installation PyTorch compatible (direct dans l'env système)
print("🔥 Installation PyTorch + CUDA 11.8...")
run_cmd('pip install --upgrade pip wheel setuptools')
run_cmd('pip install torch==2.1.0 torchvision==0.16.0 torchaudio==2.1.0 --index-url https://download.pytorch.org/whl/cu118')

# 3. Installation xformers compatible
print("⚡ Installation xformers optimisé...")
run_cmd('pip install xformers==0.0.22.post4 --index-url https://download.pytorch.org/whl/cu118')

# 4. Résolution des conflits de versions
print("🔧 Résolution des conflits de dépendances...")
run_cmd('pip install numpy==1.24.3 --force-reinstall')
run_cmd('pip install pillow==10.0.1 --force-reinstall')
run_cmd('pip install opencv-python-headless==4.8.1.78')

# 5. Clone ComfyUI
print("📥 Téléchargement ComfyUI...")
if os.path.exists('/content/ComfyUI'):
    run_cmd('rm -rf /content/ComfyUI')
run_cmd('git clone https://github.com/comfyanonymous/ComfyUI.git')

os.chdir('/content/ComfyUI')

# 6. Installation dépendances ComfyUI
print("📚 Installation dépendances ComfyUI...")
run_cmd('pip install -r requirements.txt')
run_cmd('pip install scipy scikit-image transformers accelerate safetensors')

# 7. Installation ComfyUI Manager
print("🔌 Installation ComfyUI Manager...")
manager_path = '/content/ComfyUI/custom_nodes/ComfyUI-Manager'
if os.path.exists(manager_path):
    run_cmd(f'rm -rf {manager_path}')
run_cmd(f'git clone https://github.com/ltdrdata/ComfyUI-Manager.git {manager_path}')

# 8. Installation nodes essentiels
print("🔌 Installation des custom nodes essentiels...")
custom_nodes_dir = Path('/content/ComfyUI/custom_nodes')
custom_nodes_dir.mkdir(exist_ok=True)

essential_nodes = {
    'comfyui_controlnet_aux': 'https://github.com/Fannovel16/comfyui_controlnet_aux.git',
    'ComfyUI_IPAdapter_plus': 'https://github.com/cubiq/ComfyUI_IPAdapter_plus.git',
    'ComfyUI-AnimateDiff-Evolved': 'https://github.com/Kosinkadink/ComfyUI-AnimateDiff-Evolved.git',
    'ComfyUI-VideoHelperSuite': 'https://github.com/Kosinkadink/ComfyUI-VideoHelperSuite.git',
    'was-node-suite-comfyui': 'https://github.com/WASasquatch/was-node-suite-comfyui.git'
}

for node_name, node_url in essential_nodes.items():
    node_path = custom_nodes_dir / node_name
    if node_path.exists():
        run_cmd(f'rm -rf {node_path}')
    print(f"📥 Clonage {node_name}...")
    run_cmd(f'git clone {node_url} {node_path}', check=False)

# 9. Installation des dépendances des custom nodes
print("🔧 Installation dépendances des custom nodes...")
for node_dir in custom_nodes_dir.iterdir():
    if node_dir.is_dir() and (node_dir / 'requirements.txt').exists():
        print(f"📦 Installation deps pour {node_dir.name}...")
        run_cmd(f'pip install -r {node_dir}/requirements.txt', check=False)

run_cmd('pip install insightface==0.7.3', check=False)
run_cmd('pip install onnxruntime-gpu', check=False)
run_cmd('pip install mediapipe==0.10.8', check=False)

# 10. Création des dossiers de modèles
print("📁 Création des dossiers de modèles...")
models_base = Path('/content/ComfyUI/models')
model_dirs = [
    'checkpoints', 'controlnet', 'ipadapter', 'vae',
    'clip_vision', 'animatediff_models', 'animatediff_motion_lora',
    'upscale_models', 'embeddings'
]

for model_dir in model_dirs:
    (models_base / model_dir).mkdir(parents=True, exist_ok=True)

# 11. Téléchargement modèles essentiels
print("🎨 Téléchargement des modèles essentiels...")

def download_model(url, path, description):
    if Path(path).exists():
        print(f"✅ {description} déjà présent")
        return
    print(f"📥 Téléchargement {description}...")
    try:
        run_cmd(f'wget -q --show-progress -O "{path}" "{url}"')
        print(f"✅ {description} téléchargé")
    except:
        print(f"❌ Échec téléchargement {description}")

# Modèles essentiels
models_to_download = [
    {
        'url': 'https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/resolve/main/sd_xl_base_1.0.safetensors',
        'path': '/content/ComfyUI/models/checkpoints/sd_xl_base_1.0.safetensors',
        'desc': 'SDXL Base Model (6.9GB)'
    },
    {
        'url': 'https://huggingface.co/diffusers/controlnet-depth-sdxl-1.0/resolve/main/diffusion_pytorch_model.safetensors',
        'path': '/content/ComfyUI/models/controlnet/controlnet_depth_sdxl.safetensors',
        'desc': 'ControlNet Depth SDXL'
    },
    {
        'url': 'https://huggingface.co/h94/IP-Adapter/resolve/main/sdxl_models/ip-adapter_sdxl.safetensors',
        'path': '/content/ComfyUI/models/ipadapter/ip-adapter_sdxl.safetensors',
        'desc': 'IP-Adapter SDXL'
    },
    {
        'url': 'https://huggingface.co/h94/IP-Adapter/resolve/main/models/image_encoder/model.safetensors',
        'path': '/content/ComfyUI/models/clip_vision/CLIP-ViT-H-14-laion2B-s32B-b79K.safetensors',
        'desc': 'CLIP Vision Encoder'
    },
    {
        'url': 'https://huggingface.co/guoyww/animatediff/resolve/main/mm_sd_v15_v2.ckpt',
        'path': '/content/ComfyUI/models/animatediff_models/mm_sd_v15_v2.ckpt',
        'desc': 'AnimateDiff Motion Model'
    }
]

for model_info in models_to_download:
    download_model(model_info['url'], model_info['path'], model_info['desc'])

# 12. Configuration Cloudflare tunnel
print("🌐 Configuration du tunnel Cloudflare...")
if not os.path.exists('/content/cloudflared'):
    run_cmd('wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O /content/cloudflared')
    run_cmd('chmod +x /content/cloudflared')

# 13. Création du script de lancement CORRIGÉ
print("🚀 Création du script de lancement corrigé...")
launch_script = '''#!/bin/bash
cd /content/ComfyUI

echo "🔄 Nettoyage des processus existants..."
pkill -f cloudflared 2>/dev/null || true
pkill -f "python main.py" 2>/dev/null || true
sleep 2

echo "🌐 Démarrage du tunnel Cloudflare..."
nohup /content/cloudflared tunnel --url http://127.0.0.1:8188 > /tmp/cloudflared.log 2>&1 &

# Attente du démarrage du tunnel
echo "⏳ Attente du tunnel Cloudflare..."
TUNNEL_PID=$!
sleep 5

# Vérification que cloudflared fonctionne
if ! ps -p $TUNNEL_PID > /dev/null 2>&1; then
    echo "❌ Échec du démarrage du tunnel"
    echo "📋 Logs cloudflared:"
    cat /tmp/cloudflared.log
    exit 1
fi

# Récupération de l'URL du tunnel depuis les logs
TUNNEL_URL=""
for i in {1..15}; do
    # Chercher l'URL dans les logs cloudflared
    TUNNEL_URL=$(grep -o 'https://[^[:space:]]*trycloudflare.com' /tmp/cloudflared.log 2>/dev/null | head -1)

    if [ ! -z "$TUNNEL_URL" ]; then
        echo "✅ Tunnel actif: $TUNNEL_URL"
        break
    fi

    echo "⏳ Recherche URL tunnel... ($i/15)"
    sleep 2
done

if [ -z "$TUNNEL_URL" ]; then
    echo "❌ Impossible de récupérer l'URL du tunnel"
    echo "📋 Contenu des logs cloudflared:"
    cat /tmp/cloudflared.log
    echo ""
    echo "🔧 Le tunnel fonctionne peut-être. Vérifiez manuellement les logs."
else
    echo "🎉 ComfyUI sera accessible sur: $TUNNEL_URL"
    echo "📋 Copiez cette URL pour accéder à ComfyUI"
fi

# Attendre encore un peu que le tunnel soit bien établi
sleep 3

echo "🚀 Démarrage de ComfyUI..."
echo "📱 Interface sera disponible dans quelques secondes..."

# Démarrage de ComfyUI avec logs
python main.py --listen 0.0.0.0 --port 8188 --dont-print-server 2>&1 | tee /tmp/comfyui.log
'''

with open('/content/launch_comfyui.sh', 'w') as f:
    f.write(launch_script)
run_cmd('chmod +x /content/launch_comfyui.sh')

# 14. Création d'un script de vérification du statut
status_script = '''#!/bin/bash
echo "📊 Statut des services ComfyUI"
echo "=" * 40

echo "🌐 Tunnel Cloudflare:"
if pgrep -f cloudflared > /dev/null; then
    echo "  ✅ Cloudflared actif"
    TUNNEL_URL=$(grep -o 'https://[^[:space:]]*trycloudflare.com' /tmp/cloudflared.log 2>/dev/null | head -1)
    if [ ! -z "$TUNNEL_URL" ]; then
        echo "  🔗 URL: $TUNNEL_URL"
    else
        echo "  ⚠️  URL non trouvée dans les logs"
    fi
else
    echo "  ❌ Cloudflared non actif"
fi

echo ""
echo "🎨 ComfyUI:"
if pgrep -f "python main.py" > /dev/null; then
    echo "  ✅ ComfyUI actif sur port 8188"
else
    echo "  ❌ ComfyUI non actif"
fi

echo ""
echo "📋 Logs récents:"
echo "Cloudflared (dernières 5 lignes):"
tail -5 /tmp/cloudflared.log 2>/dev/null || echo "  Pas de logs cloudflared"
echo ""
echo "ComfyUI (dernières 3 lignes):"
tail -3 /tmp/comfyui.log 2>/dev/null || echo "  Pas de logs ComfyUI"
'''

with open('/content/check_status.sh', 'w') as f:
    f.write(status_script)
run_cmd('chmod +x /content/check_status.sh')

# 15. Patch de compatibilité Pillow
print("🔧 Application des patchs de compatibilité...")
patch_cmd = '''
find /content/ComfyUI -name "*.py" -type f -exec grep -l "Image\\.LINEAR\\|Image\\.LANCZOS" {} \\; 2>/dev/null | while read file; do
    if [ -f "$file" ]; then
        sed -i 's/Image\\.LINEAR/Image.Resampling.BILINEAR/g' "$file" 2>/dev/null || true
        sed -i 's/Image\\.LANCZOS/Image.Resampling.LANCZOS/g' "$file" 2>/dev/null || true
    fi
done
'''
run_cmd(patch_cmd, check=False)

print("✅ Installation terminée avec succès!")
print("=" * 60)
print()
print("🚀 LANCEMENT:")
print("   Dans une nouvelle cellule, exécutez:")
print("   !bash /content/launch_comfyui.sh")
print()
print("🔍 VÉRIFICATION STATUT:")
print("   Pour vérifier si tout fonctionne:")
print("   !bash /content/check_status.sh")
print()
print("📁 DOSSIERS IMPORTANTS:")
print("   • ComfyUI: /content/ComfyUI")
print("   • Modèles: /content/ComfyUI/models/")
print("   • Input: /content/ComfyUI/input/")
print("   • Output: /content/ComfyUI/output/")
print()
print("🐛 DÉPANNAGE:")
print("   • Logs Cloudflared: cat /tmp/cloudflared.log")
print("   • Logs ComfyUI: cat /tmp/comfyui.log")
print("   • Redémarrer: !bash /content/launch_comfyui.sh")
print()
print("💡 CONSEILS:")
print("   • Attendez ~30 secondes après le lancement")
print("   • L'URL tunnel s'affiche dans les logs")
print("   • Utilisez ComfyUI Manager pour plus de nodes")
print("   • Uploadez vos images dans /content/ComfyUI/input/")

In [ ]:
   !bash /content/launch_comfyui.sh


In [ ]:
# Dans une nouvelle cellule Colab, exécutez :
!cat /tmp/cloudflared.log


In [ ]:
!cd /content/ComfyUI && /content/cloudflared tunnel --url http://127.0.0.1:8188 &

In [ ]:
!cd /content/ComfyUI && python main.py --listen 0.0.0.0 --port 8188

In [ ]:
!cat /tmp/cloudflared.log | grep trycloudflare.com

In [ ]:
!/content/cloudflared tunnel --url http://127.0.0.1:8188

In [ ]:
!pkill -f "python main.py" 2>/dev/null || true
!pkill -f "cloudflared" 2>/dev/null || true

In [ ]:
!cd /content/ComfyUI && nohup python main.py --listen 0.0.0.0 --port 8188 > /tmp/comfyui.log 2>&1 &

In [ ]:
!pkill -f "python main.py" 2>/dev/null || true
!pkill -f "cloudflared" 2>/dev/null || true
import time; time.sleep(2)

In [ ]:
import subprocess
import os

# Changer vers le dossier ComfyUI
os.chdir('/content/ComfyUI')

# Démarrer ComfyUI directement
print("🚀 Démarrage de ComfyUI...")
!python main.py --listen 0.0.0.0 --port 8188 --dont-print-server &

In [ ]:
# Vérifier les processus actifs
!ps aux | grep "python main.py" | grep -v grep

# Vérifier les logs si besoin
!tail -n 10 /tmp/comfyui_stdout.log

In [ ]:
import requests
try:
    response = requests.get('http://localhost:8188', timeout=5)
    print("✅ ComfyUI actif et répond")
    print("🚀 Prêt pour le tunnel Cloudflare")
except Exception as e:
    print(f"❌ Problème: {e}")

In [ ]:
!/content/cloudflared tunnel --url http://127.0.0.1:8188